In [1]:
import math

import geopandas as gpd
import laspy
import numpy as np
import startinpy

import raster
from feature.utils import timing

In [2]:
class PointCloud:
    @timing
    def __init__(self, filename: str) -> None:
        with laspy.open(filename) as f:
            self.las = f.read()

    def __len__(self) -> int:
        return len(self.las.points)

    def __getitem__(self, key: int):
        ...


# FIXME: Rewrite this function as a PointCloud method.
@timing
def to_gdf(pc: PointCloud) -> gpd.GeoDataFrame:
    return gpd.GeoDataFrame({"id": np.arange(len(pc)), "geometry": gpd.points_from_xy(pc.las.x, pc.las.y)},
                            crs="EPSG:28992")


# FIXME: Rewrite this function as a PointCloud method.
@timing
def intersect(pc: PointCloud, objs: gpd.GeoDataFrame) -> np.ndarray:
    return to_gdf(pc).overlay(objs)["id_1"].to_numpy()


# FIXME: Rewrite this function as a PointCloud method.
@timing
def triangulate(dt: startinpy.DT, pts: laspy.PackedPointRecord, sc: str = "z") -> None:
    dt.insert(np.vstack((pts.x, pts.y, pts[sc])).transpose())

In [3]:
# class BufferableGeoDataFrame(gpd.GeoDataFrame):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__()
# 
#     def buffer(self, *args, **kwargs) -> None:
#         ...

@timing
def __init__(filename: str) -> gpd.GeoDataFrame:
    return gpd.read_file(filename)


@timing
# TODO: Review the various buffer options.
# TODO: Rewrite this function as an appropriate class method.
def buffer(objs: gpd.GeoDataFrame, dist: float = 10) -> None:
    objs["geometry"] = objs["geometry"].buffer(dist)

## Step 1
Load the point cloud into memory.

In [4]:
pc = PointCloud("../aula.LAZ")

func __init__ :-> 275 ms


## Step 2
Load the building footprints into memory.

In [5]:
fps = __init__("C:/Documents/RoofSense/data/temp/9-284-556.buildings.gpkg")

func __init__ :-> 182 ms


## Step 3
Buffer the footprints by 10 m.

NOTE: The buffer distance is selected such that most if not all the natural neighbors of 
      the points along the footprint edges are included in the subsequent steps.

In [6]:
buffer(fps)

func buffer :-> 64 ms


## Step 3
Intersect the point cloud with the buffers.

In [7]:
ids = intersect(pc, fps)

func to_gdf :-> 2320 ms
func intersect :-> 33488 ms


Filter the point cloud and detach the point records which are required in the subsequent
steps.

In [8]:
# FIXME: Integrate this operation into the intersection step.
# TODO: Overwrite the original point cloud?
pts = pc.las.points[ids]
pts = np.vstack((pts.x, pts.y,  
                 # NOTE: The point elevation must be used instead of e.g., reflectance
                 #       because it is required in the subsequent steps.
                 pts.z)).transpose()

## Step 4
Compute the Delaunay triangulation of the remaining points.

In [9]:
dt = startinpy.DT()
# NOTE: The snap tolerance cannot be set to zero (i.e., disabled) so the nearest 
#       distinct value is used.
dt.snap_tolerance = math.ulp(0)

# FIXME: Enrich this comment.
# Maintain a list the order of the "duplicate" vertices in the DT and the PC.
# FIXME: Think of better variable names.
b_idx = {}
# NOTE: This lookup table serves as a link between the DT and the PC and can thus be
#       used to query all vertex attributes given its ID in the DT.
g_idx = {}

# FIXME: Think of better variable names.
cnd_id: int  # The ID of a candidate vertex before it is inserted into the DT.
tnt_id: int  # The ID of a candidate vertex assuming it will be inserted into the DT.
act_id: int  # The ID of a candidate vertex in the DT.

tnt_id = 1
for cnd_id, cnd_pt in enumerate(pts):
    act_id = dt.insert_one_pt(*cnd_pt)
    if tnt_id == act_id:
        g_idx[act_id] = cnd_id
        tnt_id += 1
    else:
        b_idx[act_id] = cnd_id

# Cache the result because this array is compiled on demand.
dt2_pts = dt.points
for act_id, cnd_id in b_idx.items():
    if dt2_pts[act_id][2] > pts[cnd_id][2]:
        dt.remove(act_id)
        dt.insert_one_pt(*pts[cnd_id])
        # Replace the previous ID of the vertex in the PC with the current one.
        g_idx[act_id] = cnd_id

Confirm that the resulting lookup table is correct.

In [10]:
np.allclose(dt.points[1:], pts[list(g_idx.values())])

True

## Step 5
 Estimate the slope field of the corresponding network.


In [11]:
# TODO

## Step 6
 Rasterize the slope and reflectance fields of the corresponding network.

In [12]:
# FIXME: Promote this constant to an environment variable.
GRID_SIZE = 0.25

# FIXME: Rewrite the following cells as Raster methods.
# Initialize the raster grid.
grid = raster.Raster(GRID_SIZE, dt.get_bbox())

# Gather the center coordinates of the grid cells.
cnts = []
for row in range((grid.len_y - 1), -1, -1):
    y = grid.bbox[1] + (row * GRID_SIZE) + (0.5 * GRID_SIZE)
    for col in range(grid.len_x):
        x = grid.bbox[0] + (col * GRID_SIZE) + (0.5 * GRID_SIZE)
        cnts.append([x, y])

# Interpolate the field values at the cell centers.
vals = dt.interpolate({"method": "Laplace"}, cnts)

# Populate the raster.
grid._Raster__data = vals.reshape((grid.len_y, grid.len_x))

# Save the raster.
grid.save("../aula_z.tiff")